In [1]:
# Importing Required Libraries
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.models import Sequential


In [2]:

def extract_feature(file_name):
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)
        return mfccs_processed
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        print(e)
        return None

    

In [3]:
if __name__ == '__main__':
    # Define the path to the dataset
    data_path = "C:/Users/ADMIN/Downloads/Data/genres_original"

    # Define the number of MFCCs to extract
    num_mfcc = 40

    # Define the number of audio files to read per genre
    num_files_per_genre = 100

    # Define the list of genres to use
    genres = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

    # Initialize empty arrays to store features and labels
    features = None
    labels = None

    # Loop through each genre
    for genre in genres:
        # Define the path to the genre folder
        genre_path = os.path.join(data_path, genre)

        # Loop through the first N audio files in the genre folder
        for i, file_name in enumerate(os.listdir(genre_path)[:num_files_per_genre]):
            # Construct the full path to the audio file
            file_path = os.path.join(genre_path, file_name)

            # Extract the MFCCs for the audio file
            mfccs = extract_feature(file_path)

            # If the MFCCs were successfully extracted, add them to the feature matrix and label vector
            if mfccs is not None:
                if features is None:
                    features = mfccs.reshape(1, -1)
                    labels = np.array([genre])
                else:
                    features = np.vstack([features, mfccs.reshape(1, -1)])
                    labels = np.append(labels, genre)

    # Encoding Labels to Integers
    le = LabelEncoder()
    labels = le.fit_transform(labels)

    # Splitting data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

    # Initialize the neural network model
    model = Sequential()

    # Add layers to the model
    model.add(Dense(units=128, activation='relu', input_shape=(num_mfcc,)))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=len(genres), activation='softmax'))
    
    # Compiling the Model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Training the Model on the Training Data
    model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        batch_size=32,
        epochs=100
    )

    # Evaluating the Model on the Testing Data
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy:", accuracy)



Epoch 1/100
25/25 [==============================] - 1s 9ms/step - loss: 5.0144 - accuracy: 0.1963 - val_loss: 2.4844 - val_accuracy: 0.2450
Epoch 2/100
25/25 [==============================] - 0s 3ms/step - loss: 2.1455 - accuracy: 0.3025 - val_loss: 1.9588 - val_accuracy: 0.3200
Epoch 3/100
25/25 [==============================] - 0s 3ms/step - loss: 1.6757 - accuracy: 0.4087 - val_loss: 1.6661 - val_accuracy: 0.3750
Epoch 4/100
25/25 [==============================] - 0s 3ms/step - loss: 1.5243 - accuracy: 0.4725 - val_loss: 1.5631 - val_accuracy: 0.4950
Epoch 5/100
25/25 [==============================] - 0s 3ms/step - loss: 1.3955 - accuracy: 0.5025 - val_loss: 1.5006 - val_accuracy: 0.4850
Epoch 6/100
25/25 [==============================] - 0s 3ms/step - loss: 1.2857 - accuracy: 0.5462 - val_loss: 1.5778 - val_accuracy: 0.4750
Epoch 7/100
25/25 [==============================] - 0s 3ms/step - loss: 1.3108 - accuracy: 0.5300 - val_loss: 1.5596 - val_accuracy: 0.4950
Epoch 8/100
2

Epoch 59/100
25/25 [==============================] - 0s 4ms/step - loss: 0.1128 - accuracy: 0.9775 - val_loss: 2.1241 - val_accuracy: 0.6350
Epoch 60/100
25/25 [==============================] - 0s 4ms/step - loss: 0.1184 - accuracy: 0.9750 - val_loss: 2.0076 - val_accuracy: 0.6150
Epoch 61/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0911 - accuracy: 0.9800 - val_loss: 2.0142 - val_accuracy: 0.6200
Epoch 62/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0958 - accuracy: 0.9762 - val_loss: 2.0303 - val_accuracy: 0.6150
Epoch 63/100
25/25 [==============================] - 0s 4ms/step - loss: 0.0763 - accuracy: 0.9875 - val_loss: 1.9876 - val_accuracy: 0.6450
Epoch 64/100
25/25 [==============================] - 0s 5ms/step - loss: 0.0792 - accuracy: 0.9887 - val_loss: 1.9373 - val_accuracy: 0.6400
Epoch 65/100
25/25 [==============================] - 0s 4ms/step - loss: 0.0863 - accuracy: 0.9862 - val_loss: 2.1440 - val_accuracy: 0.6150
Epoch 

In [11]:
# Save the trained model
model.save('my_model.h5')

# Load the saved model
loaded_model = keras.models.load_model('my_model.h5')

# Predict the genre of an audio file
mfccs = extract_feature('C:/Users/ADMIN/Downloads/Data/audio_file.wav')

input_data = mfccs.reshape(1, -1)
# print(input_data)
predicted_genre = loaded_model.predict(input_data)

# Extract genre name from probabilities
genres = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]
predicted_genre_index = np.argmax(predicted_genre)
predicted_genre_name = genres[predicted_genre_index]
print('Predicted genre: ', predicted_genre_name)


1/1 [==============================] - 0s 42ms/step
Predicted genre:  rock
